In [317]:
import yfinance as yf
import pandas as pd
import numpy as np
import os

# symbols = ["AAPL", "MSFT", "GOOGL", "AMZN", "TSLA", "META", "AMZN"]




def get_symbol(symbol,start_date,end_date):
    filename = f"{symbol}_{start_date}_to_{end_date}.csv"
    if os.path.exists(filename):
        print("CSV exists, skipping download:", filename)
    else:
        data = yf.download(symbol, start=start_date, end=end_date)
        data.to_csv(filename)
        print("Saved CSV:", filename)
    return filename

        
symbol = "AAPL"
start_date = "2014-10-26"  
end_date = "2024-10-26"
filename = get_symbol(symbol,start_date,end_date)
# load CSV back into `data`
data = pd.read_csv(filename, index_col=0, parse_dates=True)
print("Loaded CSV:", filename, "rows:", len(data))

# Ensure Close column is numeric: coerce non-numeric values to NaN then drop them
if "Close" in data.columns:
    data["Close"] = pd.to_numeric(data["Close"], errors="coerce")
    dropped = data["Close"].isna().sum()
    if dropped > 0:
        print(f"Dropped {dropped} non-numeric Close rows")
    data = data.dropna(subset=["Close"])

def get_rsi(close: pd.Series, period: int = 14) -> pd.Series:
    """
    Compute Relative Strength Index (RSI) for a pandas Series of close prices.
    Uses an exponential moving average for gains/losses (common variant).
    """
    close = close.astype(float)
    delta = close.diff()
    up = delta.clip(lower=0.0)
    down = -delta.clip(upper=0.0)

    # exponential moving averages of gains and losses
    ma_up = up.ewm(alpha=1.0/period, adjust=False).mean()
    ma_down = down.ewm(alpha=1.0/period, adjust=False).mean()

    rs = ma_up / (ma_down + 1e-8)
    rsi = 100.0 - (100.0 / (1.0 + rs))
    return rsi

# attach RSI to dataframe (if Close exists)
if "Close" in data.columns:
    data["RSI"] = get_rsi(data["Close"])
else:
    print("Warning: 'Close' column not found in data; RSI not computed.")

# change Price to Date and remove second and third rows
data = data.iloc[2:]
data = data.reset_index().rename(columns={"Price": "Date"})
data.head()






CSV exists, skipping download: AAPL_2014-10-26_to_2024-10-26.csv
Loaded CSV: AAPL_2014-10-26_to_2024-10-26.csv rows: 2519


C:\Users\Lau\AppData\Local\Temp\ipykernel_39852\2166255425.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv(filename, index_col=0, parse_dates=True)


ValueError: could not convert string to float: 'AAPL'